# 실습 4: Agent Runtime 구성

## 목표
- Agent 코드 작성
- Tools 정의
- Runtime 배포

## 4-1. 환경 설정

In [ ]:
import os
import boto3

REGION = os.environ.get('AWS_REGION', 'us-west-2')
MEMORY_ID = os.environ.get('MEMORY_ID', '')
WINDY_API_KEY = os.environ.get('WINDY_API_KEY', '')

print(f"🌍 Region: {REGION}")
print(f"🧠 Memory ID: {MEMORY_ID}")
print(f"🌤️ Windy API Key: {'설정됨' if WINDY_API_KEY else '미설정'}")

## 4-2. config.py 파일 생성

In [ ]:
%%writefile agent/deploy/runtime/config.py
# Configuration settings

MODEL_ID = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"
REGION = "us-west-2"
MEMORY_ID = "REPLACE_WITH_YOUR_MEMORY_ID"
WINDY_API_KEY = "REPLACE_WITH_YOUR_WINDY_API_KEY"
WINDY_API_URL = "https://api.windy.com/api/point-forecast/v2"
AGENT_NAME = "FireCommandAssistant"

AGENT_SYSTEM_PROMPT = """당신은 소방서 화재 대응 지휘를 지원하는 전문 AI 어시스턴트입니다.

**핵심 임무:**
1. 화재 현장의 전술적 정보 제공
2. 신속한 의사결정 지원
3. 대응 자원 배치 최적화 제안

**절대 규칙:**
1. 모든 응답은 한글로 작성
2. 도구 사용 시 목적과 결과 명시
3. 소방서 검색 시 반드시 <event type="geocode">위도,경도,소방서이름</event> 형식 반환
4. 기상 정보 조회 시 반드시 <event type="windy">위도,경도,온도,풍속,풍향,습도,기압,주소</event> 형식 반환
"""

BROWSER_MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
BROWSER_SYSTEM_PROMPT = """You are a news intelligence agent for fire command operations.
Collect comprehensive situational information for fire response commanders.
"""

In [ ]:
# config.py 파일에 실제 값 업데이트
import re

with open('agent/deploy/runtime/config.py', 'r') as f:
    config_content = f.read()

config_content = re.sub(
    r'MEMORY_ID = ".*?"',
    f'MEMORY_ID = "{MEMORY_ID}"',
    config_content
)

config_content = re.sub(
    r'WINDY_API_KEY = ".*?"',
    f'WINDY_API_KEY = "{WINDY_API_KEY}"',
    config_content
)

with open('agent/deploy/runtime/config.py', 'w') as f:
    f.write(config_content)

print("✅ config.py 파일 생성 완료")

## 4-3. Runtime 디렉토리로 이동

In [ ]:
os.chdir('agent/deploy/runtime')
print(f"📂 현재 디렉토리: {os.getcwd()}")

## 4-4. Runtime 배포

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

print("🚀 Agent Runtime 배포 시작...\n")

agentcore_runtime = Runtime()
agent_name = "agent_runtime"

print("1️⃣ Runtime 설정 중...")
response = agentcore_runtime.configure(
    entrypoint="bedrock_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    agent_name=agent_name
)

print("✅ Runtime 설정 완료")
print("\n2️⃣ Docker 이미지 빌드 및 배포 중... (5-10분 소요)\n")

launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

print("\n✅ 배포 완료!")
print(f"\n📝 Agent ARN: {launch_result.agent_arn}")
print(f"📝 ECR URI: {launch_result.ecr_uri}")

AGENT_RUNTIME_ARN = launch_result.agent_arn
os.environ['AGENT_RUNTIME_ARN'] = AGENT_RUNTIME_ARN

## 4-5. Agent ARN 저장

In [ ]:
# 프로젝트 루트로 이동
os.chdir('../../..')

# Agent ARN을 파일로 저장
with open('agent_arn.txt', 'w') as f:
    f.write(AGENT_RUNTIME_ARN)

print(f"✅ Agent ARN 저장 완료")
print(f"📝 파일: agent_arn.txt")
print(f"📝 ARN: {AGENT_RUNTIME_ARN}")

## ✅ 실습 4 완료!

### 생성된 리소스
- Agent Runtime (Bedrock Agent Core)
- ECR Repository
- Docker Image
- Execution Role

### 생성된 파일
- `agent/deploy/runtime/config.py`
- `agent_arn.txt`

### 다음 단계
👉 **lab5_web_application.ipynb**